# Linear Algebra I

Linear algebra is a core topic in modern applied mathematics. Essentially every important method in statistics, data science, and machine learning is built on linear algebra. Indeed, deep neural networks, which we will discuss shortly, are built on a foundation of matrix multiplication coupled with simple nonlinear functions. 

<figure class="image" style="width:30%">
  <img src="https://imgs.xkcd.com/comics/machine_learning_2x.png" alt="">
  <figcaption><i></i></figcaption>
</figure>

In this lecture, we'll see how to perform several important operations in linear algebra using our good friend, Numpy. These operations include: 

- Matrix multiplication. 
- Exact and approximate solutions to linear systems. 
- Singular value and eigenvalue-eigenvector decompositions. 

Along the way, we'll show several examples from statistics and applied mathematics, including simulation of partial differential equations; least-squares regression; and image compression. 

This is probably the lecture in which things will get "the most mathematical." Comfort with concepts from Math 33A or equivalent will be helpful. If you're not familiar with these concepts, that's ok -- feel free to ask questions. We'll all get through this just fine. 

In [ ]:
# no fancy packages this time! just our good friends numpy and matplotlib
import numpy as np
from matplotlib import pyplot as plt
np.random.seed(1234)

## Basic Matrix Operations

A *matrix* is a two-dimensional array of numbers. 

In [ ]:
# random matrix data to play with


Matrices admit several standard operations, including: 

In [ ]:
# scalar multiplication


In [ ]:
# transposition


In [ ]:
# application of transposition: a "symmetrized" version of A
# symmetric matrices satisfy A = A.T


Inversion is an especially important matrix operation. The inverse $\mathbf{A}^{-1}$ of a square matrix $\mathbf{A}$ satisfies $\mathbf{A}\mathbf{A}^{-1} = \mathbf{I}$, where $\mathbf{I}$ is the identity matrix. We'll see how to multiply matrices and check this in a sec. 

In [ ]:
# inverse of A


## Matrix multiplication

In [ ]:
# random vector


In [ ]:
# matrix-vector product


# modern, convenient syntax -- same effect


In [ ]:
# random matrix


# matrix-matrix product (same as A.dot(B))


In [ ]:
# checking our inverse from earlier
# observe--finite precision arithmetic! 


In [ ]:
 # looks like the identity matrix

In [ ]:
# identity matrix


# check the result to within machine precision, entrywise


In [ ]:
# aggregates the above


## Application: Simulating Heat Diffusion

Matrix multiplication provides a simple way to simulate 1-dimensional partial differential equations in discrete time. For example, the 1-d heat equation reads

$$\frac{\partial f(x, t)}{\partial t} = \frac{\partial^2 f}{\partial x^2 }\;.$$

In a discrete approximation, we can write this as 

$$f(x_i, t + 1) - f(x_i, t) \approx \epsilon\left[f(x_{i+1}, t) - 2f(x_i, t) + f(x_{i-1}, t)\right]\;,$$

where $\epsilon$ is a small positive number that controls the timescale of the approximation. 

We can write the righthand side of this equation as a matrix-vector product:

- Let $\mathbf{v}(t)$ be the values of $f(\mathbf{x}, t)$ -- that is, $v_i = f(x_i)$. 
- Let $\mathbf{A}$ be the *transition operator*: 

$$
\mathbf{A} = \left[\begin{matrix}
    -2 &  1 & 0 & \cdots& 0& 0 & 0\\
     1 & -2 & 1 & \cdots & 0& 0 &  0\\
     0 & 1 & -2 & \cdots & 0& 0 &  0\\
     \vdots & \vdots &\vdots & \ddots &  \vdots & \vdots & \vdots \\ 
     0  & 0  & 0  & \cdots   & -2     & 1 & 0\\
     0  & 0  & 0  & \cdots   & 1     & -2 & 1\\
     0 & 0  & 0  & \cdots & 0     & 1     & -2 \\
\end{matrix}\right]
$$

This transition operator has the property that $[\mathbf{A}\mathbf{v}]_i$ is equal to the righthand side of the discrete approximation, for $i = 2,\ldots,n-1$. That is, we can write 

$$
\mathbf{v}(t+1) = \mathbf{v}(t) + \epsilon \mathbf{A}\mathbf{v}(t) = (\mathbf{I} + \epsilon\mathbf{A})\mathbf{v}(t)
$$

Note that there are issues at the boundary (i.e. where $i = 1$ or $i = n$), and further modeling decisions must be made in order to handle these. In the transition operator above, we are effectively allowing heat to escape at the boundaries. 

To simulate heat diffusion in Python, we can just build this transition operator as a matrix (`numpy` array) and then iterate this update. 

In [ ]:
# size of simulation
n = 201

# Construct A using the handy np.diag() function


In [ ]:
# construct initial condition: 1 unit of heat at midpoint. 


In [ ]:
# simulate diffusion and plot, time intervals of 50


We observe the bell-shaped curve (Gaussian distribution) characteristic of 1d diffusion, just as we'd expect. Note that once we constructed the discretized approximation, we were able to perform the simulation in Python purely via linear algebra! 

## Solving Linear Equations

One of the most fundamental tasks in applied mathematics is solving linear systems of the form 

$$\mathbf{A}\mathbf{x} = \mathbf{b}\;,$$

where $\mathbf{A} \in \mathbb{R}^{n \times m}$, $\mathbf{x} \in \mathbb{R}^{m}$, and $\mathbf{b} \in \mathbb{R}^{n}$. This equation represents a set of linear relationships between variables, a single one of which looks like this: 

$$
a_{i1}x_1 + a_{i2}x_2 + \cdots + a_{im}x_m = b_i\;.
$$

Collectively, the equations in a linear system define a "flat space" called a *affine subspace* of $\mathbb{R}^m$.  

> 1. When $\mathbf{A}$ is square and of full rank (determinant nonzero), this equation has a unique solution. 
> 2. When $\mathbf{A}$ is not square or not of full rank, then this equation may have either 0 or infinitely many solutions. 

In Case 1 ("the good case"), we can use a simple built-in `numpy` method: `np.linalg.solve`. 

In [ ]:
# solve A@x = b for x



In [ ]:
# manual approach (not as efficient)
# compute the inverse explicitly and 
# premultiply by it


In Case 2 ("the bad case"), in which the matrix is either not of full rank or not square, we need to resort to subtler means. Suppose that the matrix $\mathbf{A}$ has more rows than columns: 

In this case, there usually are no exact solutions to the equation $\mathbf{A}\mathbf{x} = \mathbf{b}$. If we try the method from before, `numpy` will complain at us: 

One of the most common ways to approach this kind of problem is to compute the *least-squares approximation*, which is the minimizer $\mathbf{x}$ of the function 

$$f(\mathbf{x}) = \lVert \mathbf{A}\mathbf{x} - \mathbf{b} \rVert^2\; = \sum_i \left(b_i - \sum_j a_{ij} x_{j}\right)^2.$$

Note that, if $\mathbf{b} \in \text{range}(\mathbf{A})$; that is, if $\mathbf{b}$ is one of those lucky values such that $\mathbf{A}\mathbf{x} = \mathbf{b}$ does indeed have an exact solution, then we can choose $\mathbf{x}$ such that $f(\mathbf{x}) = 0$ by finding the exact solution. 

Otherwise, we need to satisfy ourself with an approximation, i.e. a value $\mathbf{x}$ such that $f(\mathbf{x}) > 0$. 

In [ ]:
# compute the solution x, error f(x), rank of A, and singular values of A


In [ ]:
 # approximate solution

## Application: Linear Regression, Several Ways

Actually, the problem of minimizing $f(\mathbf{x}) = \lVert \mathbf{A}\mathbf{x} - \mathbf{b} \rVert^2$ has a special name in statistics -- it's linear regression! Specifically, it's *orderinary least-squares multvariate linear regression*.  It's usually written like this: 

$$f(\beta) = \lVert \mathbf{X}\beta - \mathbf{y} \rVert^2\;,$$

where $\mathbf{X}$ is the matrix of observations of the dependent variables, and $\mathbf{y}$ is the vector of observations of the dependent variable. $\beta$ is the vector of coefficients, and it's the thing that we want to estimate. We do this by finding an estimate $\hat{\beta}$ that makes $f(\hat{\beta})$ small. 

By the way, if you are familiar with the topic of *loss functions* in machine learning, this function $f$ is called the *square-error loss* for estimating $\mathbf{y}$, and is probably the most important of all loss functions for regression tasks. 

Let's use least-squares approximation to perform 1d linear regression "by hand": 

In [ ]:
# formally, x needs to be 2d for this to work
# so we give it an extra dimension using reshape


This works in any number of dimensions! 

In fact, this least-squares problem has an analytic solution in terms of matrix inverses as well, given by 

$$
\hat{\beta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T \mathbf{y}\;.
$$

This matrix computation is easy to implement in Python using tools from the previous section. Rather than creating a plot, let's show how we perform multidimensional regression and recover an estimate of the coefficient vector $\beta$. 

The reason that `numpy` implements a specialized least-squares function like `lstsq` is that the analytic approach can be impractical to compute when the number of columns in `X` is large, as matrix inversion is a very slow operation. 